In [1]:
## Imports
from pymilvus import connections, DataType, Collection, FieldSchema, CollectionSchema, utility, db
from llama_index.core import StorageContext
import pandas as pd
import numpy as np
from langchain.text_splitter import RecursiveCharacterTextSplitter
from sentence_transformers import SentenceTransformer
import warnings

warnings.filterwarnings("ignore")
import dotenv
from dotenv import load_dotenv
import wikipedia
import requests
import re

load_dotenv()

C:\Users\Kiril\PycharmProjects\ibm-watsonx\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


True

In [3]:
# Credentials
import os

host = os.getenv("MILVUS_HOST", None)
port = os.getenv("MILVUS_PORT", None)
password = os.getenv("MILVUS_PASSWORD", None)
user = os.getenv("MILVUS_USER", None)
print(f"Host: {host}, Port: {port}, Password: {password}, User: {user}")

Host: 102092af-5474-4a42-8dc2-35bb05ffdd0e.cvgfjtof0l91rq0joaj0.lakehouse.appdomain.cloud, Port: 31574, Password: oLDjQJic9EgszqIbNKUCR4_pxvr1L-DGKlpxkQ7p7zJK, User: ibmlhapikey


In [4]:
# Connect to Milvus
connections.connect(user=user, password=password, host=host, port=port, secure=True)
# test connection
print(connections.list_connections())
print(connections.get_connection_addr(alias="default"))
print(db.list_database())

[('default', <pymilvus.client.grpc_handler.GrpcHandler object at 0x00000276E6F065A0>)]
{'secure': True, 'address': '102092af-5474-4a42-8dc2-35bb05ffdd0e.cvgfjtof0l91rq0joaj0.lakehouse.appdomain.cloud:31574', 'user': 'ibmlhapikey'}
['default', 'group_01', 'group_02', 'group_03']


In [ ]:
OLD_COLL = Collection("docling_helvetia")
OLD_COLL.load()

BATCH = 6000  # tune for memory/network limits

iterator = OLD_COLL.query_iterator(expr="", output_fields=["*"], batch_size=BATCH)

In [ ]:
def transform(rows):
    out = []
    for r in rows:
        dst = {k: r[k] for k in (
            "vector", "text", "metadata", "page_number", "file_name",
            "product_name", "product_year", "chapter", "company_entity")
               }
        for month, file_name in zip(product_months, file_names):
            if file_name == dst["file_name"]:
                dst["product_month"] = int(month)
                break

        out.append(dst)
    return out

In [21]:
# Set collection name
COLLECTION_NAME = "docling_helvetia"  #name
collection = Collection(COLLECTION_NAME)


# get all file names
def all_file_names(collection, batch=10000) -> set[str]:
    seen = set()
    iterator = collection.query_iterator(expr="",  # empty expr = everything
                                         output_fields=["file_name"],
                                         batch_size=batch)  # stream in blocks
    results = []
    while True:
        result = iterator.next()
        if not result:
            iterator.close()
            break
        # print(result)
        results += result
    for page in results:
        if "file_name" in page:
            seen.add(page["file_name"])

    return seen


file_names = all_file_names(collection)
print(f"Found {len(file_names)} distinct files")


Found 695 distinct files


In [28]:
# Extract from Excel
# Placeholder values - replace with actual extraction logic
product_names = []
product_years = []
product_months = []
company_entity_names = []
# try:
df = pd.read_excel("../.data/Uebersicht_AVB_formatted.xlsx", sheet_name="Helvetia", dtype={'Year': str})
for file_name in file_names:
    matching_row = df[df['File'] == file_name]
    if not matching_row.empty:
        # Get the first match (assuming filenames are unique)
        product_name = matching_row.iloc[0]['Produkt']
        split_date = str(matching_row.iloc[0]['Year']).strip()
        month, _sep, year = split_date.rpartition(".")
        product_month = int(month) if _sep else 1
        product_year = int(year) if _sep else split_date
        if product_year == "nan":
            product_year = None

        print(f"Filename: {file_name}")
        print(f"Found Product Name: {product_name}")
        print(f"Found Product Month: {product_month}")
        print(f"Found Year: {product_year}")

        product_names.append(product_name)
        product_months.append(product_month)
        product_years.append(product_year)
        company_entity_names.append("Helvetia")
    else:
        print(f"Filename '{file_name}' not found in the Excel file.")
        raise Exception(f"Filename '{file_name}' not found in the Excel file.")
# except FileNotFoundError:
#     print(f"Error: Excel file not found at '{df.path}'")
# except KeyError as e:
#     print(f"Error: Column {e} not found in the Excel file. Check column names.")
# except Exception as e:
#     print(f"An unexpected error occurred: {e}")
print(len(file_names))
assert len(file_names) == len(product_months), "Lists must have same length"
# product_month must already be a declared INT64 column
assert "product_month" in {f.name for f in collection.schema.fields}

Filename: Ergänzende Bedingungen für Vorsorgeversicherungen (Säule 3a) gültig ab 01.01.2005_DE.pdf
Found Product Name: Branche 1210
Found Product Month: 1
Found Year: 2005
Filename: 1121 09.90_DE.pdf
Found Product Name: Branche 1230
Found Product Month: 9
Found Year: 1990
Filename: 2-5712 01.05 Patria_DE.pdf
Found Product Name: Branche 1210
Found Product Month: 1
Found Year: 2005
Filename: 2-6301 03.09_DE.pdf
Found Product Name: Vorsorgeversicherung
Found Product Month: 3
Found Year: 2009
Filename: 2-0558 05.39_DE.pdf
Found Product Name: Branche 1210
Found Product Month: 5
Found Year: 1939
Filename: 12-10643 09.16_DE.pdf
Found Product Name: Garantieplan
Found Product Month: 9
Found Year: 2016
Filename: 2-0640 03.57_DE.pdf
Found Product Name: Branche 1210
Found Product Month: 3
Found Year: 1957
Filename: 120015405 11.22_DE.pdf
Found Product Name: Performanceplan
Found Product Month: 11
Found Year: 2022
Filename: 2-2092 12.94_DE.pdf
Found Product Name: Branche 1210
Found Product Month: 1

In [ ]:
# iterate over all rows in the collection
from pymilvus import Collection
collection = Collection(COLLECTION_NAME)
# collection.load()

results = []
while True:
    result = iterator.next()
    if not result:
        iterator.close()
        break
    # print(result)
    collection.insert(transform(result))

collection.flush()